In [10]:
import os
from PIL import Image
from qdrant_client import QdrantClient
from qdrant_client.models import SearchParams
from src.FeatureExtractor import FeatureExtractor


# Function to perform image search using Qdrant
def search_similar_images(query_image_path, model_name, collection_suffix, top_k=5):
    # Create a unique collection name based on the model name
    collection_name = f"image_embeddings_{collection_suffix}"

    # Connect to Qdrant client
    client = QdrantClient(url="http://localhost:6333")

    # Initialize feature extractor
    extractor = FeatureExtractor(model_name)

    # Extract feature vector from the query image
    query_vector = extractor(query_image_path)

    # Perform the search in Qdrant
    search_params = SearchParams(
        hnsw_ef=128,  # Higher values mean more accurate but slower searches
        exact=False  # Set to True for exact search, but False (approximate) is faster
    )
    
    results = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=top_k,  # Number of most similar images to retrieve
        search_params=search_params,
        with_payload=["filename", "class"]  # Retrieve filename and class info
    )

    return results


# Function to display search results
def display_results(results):
    print(f"Found {len(results)} similar images:\n")
    
    for result in results:
        filename = result.payload["filename"]
        class_label = result.payload["class"]
        score = result.score  # Similarity score

        # Load and display the image
        img = Image.open(filename)
        img.show()

        print(f"Filename: {filename}")
        print(f"Class: {class_label}")
        print(f"Similarity Score: {score}")
        print("-" * 40)


# Main function to perform image search
if __name__ == "__main__":
    # Define the query image path
    query_image_path = "train/images/tiramisu/2680.jpg"

    # Define the model and collection used to store the vectors
    model_name = "resnet34"  # Change this to the model used to store the vectors
    collection_suffix = "resnet34"

    # Search for similar images
    results = search_similar_images(query_image_path, model_name, collection_suffix, top_k=5)

    # Display search results
    display_results(results)


UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Wrong input: Vector dimension error: expected dim: 129, got 128"},"time":0.00040432}'